In [0]:
# !pip install tensorflow-gpu==2.0.0-alpha0

In [0]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, GRU, Dropout, Embedding
from music21 import converter, instrument, note, chord, pitch
from google.colab import drive
import numpy as np
import time
import glob
import pickle

In [0]:
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
data_path = "/content/gdrive/My Drive/Applied Deep Learning/Data/"

In [0]:
def get_notes(filename):
  notes = []
  midi = converter.parse(filename)
  notes_to_parse = None
  try:  # file has instrument parts
    s2 = instrument.partitionByInstrument(midi)
    notes_to_parse = s2.parts[0].recurse() 
  except:  # file has notes in a flat structure
    notes_to_parse = midi.flat.notes

  for element in notes_to_parse:
    
    new_chord = [-1] * 6 # 5 pitches + 1 duration
    
    if isinstance(element, note.Note):
      new_chord[0] = element.pitch.ps
      new_chord[5] = float(element.duration.quarterLength)
#       new_chord[5] = round(float(element.duration.quarterLength), 3)
      notes.append(new_chord)
    elif isinstance(element, chord.Chord):
      new_chord[5] = float(element.duration.quarterLength)
#       new_chord[5] = round(float(element.duration.quarterLength), 3)
      
      for k in range(len(element.pitches)):
        new_chord[k] = element.pitches[k].ps
        
        if k >= len(new_chord) - 1:
          break

      notes.append(new_chord)

  return notes


In [0]:
from magenta.music import abc_parser as abc_parser
import numpy as np

MAX_NOTES = 2500
NUM_PARAM = 5 + 1 # 5 pitches + 1 duration

# 3D numpy array: num songs x MAX_NOTES x NUM_PARAM
all_songs_data = np.empty((1, MAX_NOTES, NUM_PARAM), np.float64)

for f in glob.glob(data_path + '*.mid'):
  song_notes = get_notes(f)

  temp_data = np.empty((1, 1, NUM_PARAM), np.float64)
  notes_data = np.zeros((1, MAX_NOTES, NUM_PARAM)) - 1
  
  for elem in song_notes:
    # Convert notes into a 2D numpy array
    # The 1st dimension is 5 pitches and 1 duration
    # The 2nd dimension is the number of notes
    raw_notes = np.array([[elem]]) 
    temp_data = np.append(temp_data, raw_notes, axis=1)
  
  temp_data = temp_data[:, 1:, :]
  notes_data[0, :temp_data.shape[1],:temp_data.shape[2]] = temp_data
#   print('notes_data = ', notes_data)
  
  all_songs_data = np.append(all_songs_data, notes_data, axis=0)

  
all_songs_data = all_songs_data[1:,:,:]



For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

[[[52.         57.         -1.         -1.         -1.
    0.75      ]
  [40.         -1.         -1.         -1.         -1.
    0.75      ]
  [33.         40.         -1.         -1.         -1.
    0.75      ]
  ...
  [-1.         -1.         -1.         -1.         -1.
   -1.        ]
  [-1.         -1.         -1.         -1.         -1.
   -1.        ]
  [-1.         -1.         -1.         -1.         -1.
   -1.        ]]

 [[72.         -1.         -1.         -1.         -1.
    1.        ]
  [64.         -1.         -1.         -1.         -1.
    1.33333333]
  [36.         -1.         -1.         -1.         -1.
    2.        ]
  ...
  [-1.         -1.         -1.         -1.         -1.
   -1.        ]
  [-1.         -1.         -1.         -

In [0]:
all_songs_data.shape

(92, 2500, 6)

In [0]:
preprocessed_path = "/content/gdrive/My Drive/Applied Deep Learning/Outputs/"

In [0]:
import pickle

pickle.dump( all_songs_data, open( preprocessed_path + "all_songs_data.p", "wb" ) )

In [0]:
all_songs_data = pickle.load( open( preprocessed_path + "all_songs_data.p", "rb" ) )